### Chat With Multiple CSV

In [653]:
# Take csv or excel sheet & upload data in sqlite3
# Take User questions
# Parse questions & get relevant tables and columns (LLM)
# Get Unique noun
# Generate SQL query (LLM)
# Validate SQL Query if something wrong fix it (LLM)
# Execute SQL
#     If there is no error & SQL Query is relevent
#         Choose visualization (LLM)
#         format data for visualization (LLM)
#     Else
#         Format result (LLM)


In [654]:
# from langchain_community.llms import Ollama
# llm = Ollama(model="llama3.1:latest")
from langchain_groq import ChatGroq

llm = ChatGroq(groq_api_key="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", model="gemma2-9b-it")

In [655]:
from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()

In [656]:
## Insert data in sequalite
import os
import pandas as pd
from sqlalchemy import create_engine

# Function to read CSV/Excel and insert into SQLite database
def insert_data_to_sqlite(file_path):
    # Extract the file name without extension to use as table name
    file_name = os.path.splitext(os.path.basename(file_path))[0]
    
    # Read the data (change this to pd.read_excel() for Excel files)
    data = pd.read_csv(file_path)

    # Create a SQLite database (or connect if it already exists)
    engine = create_engine('sqlite:///lumin.db')
    
    # Insert data into the SQLite database with the table name as the file name
    data.to_sql(file_name, con=engine, if_exists='replace', index=False)
    print(f"Data from {file_path} has been inserted into the '{file_name}' table in the 'lumin.db' database.")



In [657]:
os.chdir("/Users/spandanjoshi/Desktop/LLM/lumin/backend/notebook")

In [658]:
# Specify the path to your CSV/Excel file
ecom_data = [
  "olist_customers_dataset.csv",
  "olist_geolocation_dataset.csv",
  "olist_order_items_dataset.csv",
  "olist_order_payments_dataset.csv",
  "olist_order_reviews_dataset.csv",
  "olist_orders_dataset.csv",
  "olist_products_dataset.csv",
  "olist_sellers_dataset.csv",
  "product_category_name_translation.csv"
]

# for data in ecom_data:
#     path = (f"ecommerce/{data}")
#     # 
    
#     file_data = os.path.abspath(path)
#     insert_data_to_sqlite(file_data)
#     print(file_data)


In [659]:
from sqlalchemy import inspect
from typing import List, Dict

# Assuming you have an SQLAlchemy engine created
engine = create_engine('sqlite:///lumin.db')  

def get_schemas(table_names: List[str]) -> List[Dict]:
    try:
        # Create an inspector object
        inspector = inspect(engine)

        # Initialize an array to hold the schema information for all tables
        schemas_info = []

        for table_name in table_names:
            schema_info = {
                "table_name": table_name,
                "schema": []
            }

            # Get the columns for the specified table
            columns = inspector.get_columns(table_name)

            # Collect column information
            for column in columns:
                schema_info["schema"].append({
                    "name": column['name'],
                    "type": str(column['type']),
                    "nullable": column['nullable']
                })

            # Append the schema information for the current table to the list
            schemas_info.append(schema_info)

        # Return the schema information for all tables
        return schemas_info

    except Exception as e:
        print(f"An error occurred: {e}")
        return []  # Return an empty list in case of an error

schema = get_schemas(['olist_products_dataset',"olist_orders_dataset","olist_customers_dataset","olist_order_items_dataset"])

print(schema)

[{'table_name': 'olist_products_dataset', 'schema': [{'name': 'product_id', 'type': 'TEXT', 'nullable': True}, {'name': 'product_category_name', 'type': 'TEXT', 'nullable': True}, {'name': 'product_name_lenght', 'type': 'FLOAT', 'nullable': True}, {'name': 'product_description_lenght', 'type': 'FLOAT', 'nullable': True}, {'name': 'product_photos_qty', 'type': 'FLOAT', 'nullable': True}, {'name': 'product_weight_g', 'type': 'FLOAT', 'nullable': True}, {'name': 'product_length_cm', 'type': 'FLOAT', 'nullable': True}, {'name': 'product_height_cm', 'type': 'FLOAT', 'nullable': True}, {'name': 'product_width_cm', 'type': 'FLOAT', 'nullable': True}]}, {'table_name': 'olist_orders_dataset', 'schema': [{'name': 'order_id', 'type': 'TEXT', 'nullable': True}, {'name': 'customer_id', 'type': 'TEXT', 'nullable': True}, {'name': 'order_status', 'type': 'TEXT', 'nullable': True}, {'name': 'order_purchase_timestamp', 'type': 'TEXT', 'nullable': True}, {'name': 'order_approved_at', 'type': 'TEXT', 'nu

In [694]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy import text  # Import text for raw SQL queries
from typing import List, Dict, Any

# Assuming you have an SQLAlchemy engine created
engine = create_engine('sqlite:///lumin.db')  # Example for SQLite

# Create a session factory bound to the engine
Session = sessionmaker(bind=engine)

def execute_query(state: dict) -> Dict[List,Any]:
    print("======= execute_query =======")
    query = state['sql_query']
    try:
        # Create a new session
        with Session() as session:  # Call Session() without any parameters
            # Use a text() construct for the query
            result = session.execute(text(query))  # Use text() for raw SQL
            # If the query is a SELECT statement, fetch the results
            if result.returns_rows:
                return {"query_result":[row for row in result.fetchall()]}  # Convert RowProxy to dict
            else:
                # For non-SELECT queries, commit the transaction and return an empty list
                session.commit()
                return {"query_result":[]}

    except Exception as e:
        print(f"An error occurred: {e}")
        return {"query_result":[]}  # Return an empty list in case of an error


In [661]:
# execute_query("SELECT * FROM results;")

In [662]:
import json

def simple_json_extraction(content):
    # Find the JSON part
    start = content.find('{')
    end = content.rfind('}') + 1
    
    # Extract and parse the JSON
    json_str = content[start:end]
    parsed_json = json.loads(json_str)
    
    # Return the formatted JSON
    return json.dumps(parsed_json, indent=2)

### Parse questions & get relevant tables and columns

In [663]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser

def get_parse_question(state: dict) -> dict:

    print(state)
    question = state['question']
    schema = state['schema']

    print("======= get_parse_question =======")

    prompt = ChatPromptTemplate.from_messages([
        ("system", '''
You are an expert data analyst tasked with analyzing SQL databases. Your goal is to interpret user questions, understand the provided schema, and identify relevant tables and columns.

Instructions:
1. Analyze the user question and database schema to identify relevant tables and columns.
2. Set "is_relevant" to false if the question is not applicable to the database or lacks sufficient information for an answer.
3. Focus on columns with meaningful nouns (e.g., names, entities) and exclude non-noun columns (e.g., IDs, numerical data) unless specifically relevant to the question.
4. Return the response in the following JSON format:
{{
  "is_relevant": boolean,
  "relevant_tables": [
    {{
      "table_name": "string",
      "columns": ["string"],
      "noun_columns": ["string"]
    }}
  ]
}}

Key Guidelines:
- Always verify column names against the provided schema.
- Include only existing schema column names in the "columns" and "noun_columns" lists.
- "noun_columns" shouldn't include any numeric value verify the type from schema, type must be not Int, Bigint or any type of integer value .
- Do not add query-mentioned values or entities to "columns" or "noun_columns" unless they are actual column names in the schema.
- Ensure "noun_columns" contains only valid column names from the schema, matching their exact format.
- Include in "noun_columns" only noun-based columns relevant to the question (e.g., "artist_name" for "Who are the top-selling artists?").
- Exclude numerical or ID columns from "noun_columns" unless they represent meaningful entities.
- If a term in the query matches a likely column value rather than a column name (e.g., "Brazil" in "matches where Brazil scored"), do not include it in the lists.

Example:
Question: "What is the total number of matches where the Brazil team scored more than 2 goals?"
- Do not include "Brazil team" in columns or noun_columns as it's likely a value, not a column name.
- Include relevant columns like "team_name", "goals_scored" if they exist in the schema.

    '''),
        ("human", "===Database Schema:\n{schema}\n\n===User Question:\n{question}\n\nIdentify the relevant tables and columns based on the provided information:")
    ])


    output_parser = JsonOutputParser()
    # Use the format method to create the formatted prompt
    formatted_prompt = prompt.format(schema=schema, question=question)
    
    # Invoke the LLM with the formatted prompt
    response = llm.invoke(formatted_prompt)
    
    extracted_json = simple_json_extraction(response.content)
    parsed_response = output_parser.parse(extracted_json)
    return {"parsed_question": parsed_response}


In [664]:
# parse_question = get_parse_question({"schema":schema,"question":"What is the total number of matches where the Brazil team scored more than 2 goals?"})
parse_question = get_parse_question({"schema":schema,"question":"Which product categories generate the most revenue?"})
print(parse_question)


{'schema': [{'table_name': 'olist_products_dataset', 'schema': [{'name': 'product_id', 'type': 'TEXT', 'nullable': True}, {'name': 'product_category_name', 'type': 'TEXT', 'nullable': True}, {'name': 'product_name_lenght', 'type': 'FLOAT', 'nullable': True}, {'name': 'product_description_lenght', 'type': 'FLOAT', 'nullable': True}, {'name': 'product_photos_qty', 'type': 'FLOAT', 'nullable': True}, {'name': 'product_weight_g', 'type': 'FLOAT', 'nullable': True}, {'name': 'product_length_cm', 'type': 'FLOAT', 'nullable': True}, {'name': 'product_height_cm', 'type': 'FLOAT', 'nullable': True}, {'name': 'product_width_cm', 'type': 'FLOAT', 'nullable': True}]}, {'table_name': 'olist_orders_dataset', 'schema': [{'name': 'order_id', 'type': 'TEXT', 'nullable': True}, {'name': 'customer_id', 'type': 'TEXT', 'nullable': True}, {'name': 'order_status', 'type': 'TEXT', 'nullable': True}, {'name': 'order_purchase_timestamp', 'type': 'TEXT', 'nullable': True}, {'name': 'order_approved_at', 'type': 

In [665]:
parse_question

{'parsed_question': {'is_relevant': True,
  'relevant_tables': [{'table_name': 'olist_order_items_dataset',
    'columns': ['order_id', 'product_id', 'price', 'freight_value'],
    'noun_columns': []},
   {'table_name': 'olist_products_dataset',
    'columns': ['product_id', 'product_category_name'],
    'noun_columns': ['product_category_name']}]}}

In [666]:
def get_unique_nouns(state: dict) -> dict:
    print("======= Find unique nouns in relevant tables and columns. =======")
    """Find unique nouns in relevant tables and columns."""
    # Extract parsed question details from the state
    parsed_question = state['parsed_question']
    
    # Return an empty dictionary if the question is not relevant
    if not parsed_question['is_relevant']:
        return {"unique_nouns": {}}

    # Initialize a dictionary to collect unique nouns segregated by table and column
    unique_nouns = {}

    # Loop through the relevant tables extracted from the parsed question
    for table_info in parsed_question['relevant_tables']:
        table_name = table_info['table_name']
        noun_columns = table_info['noun_columns']
        
        # Initialize a dictionary for the current table
        unique_nouns[table_name] = {}

        # If there are noun columns to process
        for column in noun_columns:
            # Construct the SQL query to select distinct values from the column
            query = f"SELECT DISTINCT `{column}` FROM `{table_name}`"
            
            with Session() as session:           
                result = session.execute(text(query))  # Use text() for raw SQL
                # If the query is a SELECT statement, fetch the results
                if result.returns_rows:
                    # Create a set for unique values for the current column
                    unique_values = set()
                    for row in result:
                        # Add each non-null value as a string to the unique values set
                        if row[0]:  # row[0] is the value of the column
                            unique_values.add(str(row[0]))

                    # Store the unique values in the dictionary under the respective table and column
                    unique_nouns[table_name][column] = unique_values

                else:
                    # For non-SELECT queries, commit the transaction
                    session.commit()

    # Return the unique nouns segregated by table and column
    return {"unique_nouns": unique_nouns}


In [667]:
unique_nouns = get_unique_nouns(parse_question)

======= Find unique nouns in relevant tables and columns. =======


In [668]:
# unique_nouns

### Generate SQL Query

In [669]:
prompt = ChatPromptTemplate.from_messages([
    ("system", '''
    You are an AI assistant that generates SQL queries based on user questions, database schema, and unique nouns found in the relevant tables. Your goal is to generate valid SQL queries that can directly answer the user's question.

    ### Instructions:
    1. Parse the user question, identify relevant tables and columns from the schema, and generate an SQL query using the correct table and column names.
    2. Ensure the SQL query answers the question using only two or three columns in the result.
    3. If there isn't enough information to generate a query, return "NOT_ENOUGH_INFO".
    4. Always enclose table and column names in backticks (`) for SQL syntax consistency.
    5. Skip rows where any column is NULL, empty (""), or contains "N/A".
    6. Use the exact spellings of nouns from the unique nouns list, but only include nouns that match actual column names in the schema.

    Here are some examples:

    1. **What is the top selling product?**
       **Type**: Simple Aggregation  
       **Answer**: 
       ```sql
       SELECT `product_name`, SUM(`quantity`) AS `total_quantity`
       FROM `sales`
       WHERE `product_name` IS NOT NULL AND `quantity` IS NOT NULL 
       AND `product_name` != "" AND `quantity` != "" 
       AND `product_name` != "N/A" AND `quantity` != "N/A" 
       GROUP BY `product_name` 
       ORDER BY `total_quantity` DESC 
       LIMIT 1```
         
    2. **What is the total revenue for each product?**
       **Type**: Revenue Calculation
       **Answer**: 
       ```sql
         SELECT `product_name`, SUM(`quantity` * `price`) AS `total_revenue`
         FROM `sales`
         WHERE `product_name` IS NOT NULL AND `quantity` IS NOT NULL 
         AND `price` IS NOT NULL AND `product_name` != "" 
         AND `quantity` != "" AND `price` != "" 
         AND `product_name` != "N/A" AND `quantity` != "N/A" 
         AND `price` != "N/A"
         GROUP BY `product_name`
         ORDER BY `total_revenue` DESC
         ```

    3. **What is the market share of each product?** 
       **Type**: Market Share Calculation
       **Answer**:
       ```sql
         SELECT `product_name`, 
         SUM(`quantity`) * 100.0 / (SELECT SUM(`quantity`) FROM `sales`) AS `market_share`
         FROM `sales`
         WHERE `product_name` IS NOT NULL AND `quantity` IS NOT NULL 
         AND `product_name` != "" AND `quantity` != "" 
         AND `product_name` != "N/A" AND `quantity` != "N/A"
         GROUP BY `product_name`
         ORDER BY `market_share` DESC
         ```
    4. **Which customers purchased the top-selling products?** 
       **Type**: Join Query
       **Answer**:
        ```sql
         SELECT `customers`.`customer_name`, `sales`.`product_name`, `sales`.`total_quantity`
         FROM `customers`
         JOIN `sales` ON `customers`.`customer_id` = `sales`.`customer_id`
         WHERE `sales`.`total_quantity` = (
             SELECT MAX(`total_quantity`) FROM `sales`
         )
        ```

    5. **Plot the distribution of income over time.**
       **Type**: Distribution Plot
       **Answer**:
         ```sql
         SELECT `income`, COUNT(*) AS `count`
         FROM `users`
         WHERE `income` IS NOT NULL AND `income` != "" AND `income` != "N/A"
         GROUP BY `income`
        ```

    6. **What is the total sales between 2021 and 2023?**
       **Type**: Date Range Query
       **Answer**:
         ```sql
         SELECT SUM(`quantity` * `price`) AS `total_sales`
         FROM `sales`
         WHERE `sale_date` BETWEEN '2021-01-01' AND '2023-12-31'
         ```
    7. **Find the total sales for each region, including customer count**
       **Type**: Complex Aggregation
       **Answer**:
         ```sql
         SELECT `regions`.`region_name`, SUM(`sales`.`quantity` * `sales`.`price`) AS `total_sales`, COUNT(DISTINCT `customers`.`customer_id`) AS `customer_count`
         FROM `sales`
         JOIN `customers` ON `sales`.`customer_id` = `customers`.`customer_id`
         JOIN `regions` ON `customers`.`region_id` = `regions`.`region_id`
         GROUP BY `regions`.`region_name`
         ORDER BY `total_sales` DESC
         ```
         
    ### Format for Results:
    - For simple queries (without labels): `[[x, y]]`
    - For queries with labels: `[[label, x, y]]`

    Just return the SQL query string based on the schema, question, and unique nouns provided.
    '''), 
    ("human", '''===Database schema: {schema}

    ===User question: {question}

    ===Relevant tables and columns: {relevant_table_column}
      

    Generate SQL query string:''')
])

    # ===Unique nouns in relevant tables: {unique_nouns}
    # This is the unique_noun format dont get confused with it :
    #   {
    #     "unique_nouns": {
    #         "table_name": {
    #             "column_name": {"noun1", "noun2",...},
    #             ...
    #         },
    #           "table_name": {
    #             "column_name": {"noun1", "noun2",...},
    #             ...
    #         },
    #          ...
    #     }
    # }


In [670]:
question = "Which product categories generate the most revenue?"
formatted_prompt = prompt.format(schema=schema, question=question, relevant_table_column=parse_question, unique_nouns=unique_nouns)

In [671]:
formatted_prompt

'System: \n    You are an AI assistant that generates SQL queries based on user questions, database schema, and unique nouns found in the relevant tables. Your goal is to generate valid SQL queries that can directly answer the user\'s question.\n\n    ### Instructions:\n    1. Parse the user question, identify relevant tables and columns from the schema, and generate an SQL query using the correct table and column names.\n    2. Ensure the SQL query answers the question using only two or three columns in the result.\n    3. If there isn\'t enough information to generate a query, return "NOT_ENOUGH_INFO".\n    4. Always enclose table and column names in backticks (`) for SQL syntax consistency.\n    5. Skip rows where any column is NULL, empty (""), or contains "N/A".\n    6. Use the exact spellings of nouns from the unique nouns list, but only include nouns that match actual column names in the schema.\n\n    Here are some examples:\n\n    1. **What is the top selling product?**\n     

In [672]:
def generete_sql_query(state: dict) -> dict:

    schema = state["schema"]
    question= state["question"]
    parsed_question = state["parsed_question"] 
    print("======= generete_sql_query =======")
    prompt = ChatPromptTemplate.from_messages([
        ("system", '''
        You are an AI assistant that generates SQL queries based on user questions, database schema, and unique nouns found in the relevant tables. Your goal is to generate valid SQL queries that can directly answer the user's question.

        ### Instructions:
        1. Parse the user question, identify relevant tables and columns from the schema, and generate an SQL query using the correct table and column names.
        2. Ensure the SQL query answers the question using only two or three columns in the result.
        3. If there isn't enough information to generate a query, return "NOT_ENOUGH_INFO".
        4. Always enclose table and column names in backticks (`) for SQL syntax consistency.
        5. Skip rows where any column is NULL, empty (""), or contains "N/A".
        6. Use the exact spellings of nouns from the unique nouns list, but only include nouns that match actual column names in the schema.

        Here are some examples:

        1. **What is the top selling product?**
        **Type**: Simple Aggregation  
        **Answer**: 
        ```sql
        SELECT `product_name`, SUM(`quantity`) AS `total_quantity`
        FROM `sales`
        WHERE `product_name` IS NOT NULL AND `quantity` IS NOT NULL 
        AND `product_name` != "" AND `quantity` != "" 
        AND `product_name` != "N/A" AND `quantity` != "N/A" 
        GROUP BY `product_name` 
        ORDER BY `total_quantity` DESC 
        LIMIT 1```
            
        2. **What is the total revenue for each product?**
        **Type**: Revenue Calculation
        **Answer**: 
        ```sql
            SELECT `product_name`, SUM(`quantity` * `price`) AS `total_revenue`
            FROM `sales`
            WHERE `product_name` IS NOT NULL AND `quantity` IS NOT NULL 
            AND `price` IS NOT NULL AND `product_name` != "" 
            AND `quantity` != "" AND `price` != "" 
            AND `product_name` != "N/A" AND `quantity` != "N/A" 
            AND `price` != "N/A"
            GROUP BY `product_name`
            ORDER BY `total_revenue` DESC
            ```

        3. **What is the market share of each product?** 
        **Type**: Market Share Calculation
        **Answer**:
        ```sql
            SELECT `product_name`, 
            SUM(`quantity`) * 100.0 / (SELECT SUM(`quantity`) FROM `sales`) AS `market_share`
            FROM `sales`
            WHERE `product_name` IS NOT NULL AND `quantity` IS NOT NULL 
            AND `product_name` != "" AND `quantity` != "" 
            AND `product_name` != "N/A" AND `quantity` != "N/A"
            GROUP BY `product_name`
            ORDER BY `market_share` DESC
            ```
        4. **Which customers purchased the top-selling products?** 
        **Type**: Join Query
        **Answer**:
            ```sql
            SELECT `customers`.`customer_name`, `sales`.`product_name`, `sales`.`total_quantity`
            FROM `customers`
            JOIN `sales` ON `customers`.`customer_id` = `sales`.`customer_id`
            WHERE `sales`.`total_quantity` = (
                SELECT MAX(`total_quantity`) FROM `sales`
            )
            ```

        5. **Plot the distribution of income over time.**
        **Type**: Distribution Plot
        **Answer**:
            ```sql
            SELECT `income`, COUNT(*) AS `count`
            FROM `users`
            WHERE `income` IS NOT NULL AND `income` != "" AND `income` != "N/A"
            GROUP BY `income`
            ```

        6. **What is the total sales between 2021 and 2023?**
        **Type**: Date Range Query
        **Answer**:
            ```sql
            SELECT SUM(`quantity` * `price`) AS `total_sales`
            FROM `sales`
            WHERE `sale_date` BETWEEN '2021-01-01' AND '2023-12-31'
            ```
        7. **Find the total sales for each region, including customer count**
        **Type**: Complex Aggregation
        **Answer**:
            ```sql
            SELECT `regions`.`region_name`, SUM(`sales`.`quantity` * `sales`.`price`) AS `total_sales`, COUNT(DISTINCT `customers`.`customer_id`) AS `customer_count`
            FROM `sales`
            JOIN `customers` ON `sales`.`customer_id` = `customers`.`customer_id`
            JOIN `regions` ON `customers`.`region_id` = `regions`.`region_id`
            GROUP BY `regions`.`region_name`
            ORDER BY `total_sales` DESC
            ```
            
        ### Format for Results:
        - For simple queries (without labels): `[[x, y]]`
        - For queries with labels: `[[label, x, y]]`

        Just return the SQL query string based on the schema, question, and unique nouns provided.
        '''), 
        ("human", '''===Database schema: {schema}

        ===User question: {question}

        ===Relevant tables and columns: {relevant_table_column}
        

        Generate SQL query string:''')
    ])

        # ===Unique nouns in relevant tables: {unique_nouns}
        # This is the unique_noun format dont get confused with it :
        #   {
        #     "unique_nouns": {
        #         "table_name": {
        #             "column_name": {"noun1", "noun2",...},
        #             ...
        #         },
        #           "table_name": {
        #             "column_name": {"noun1", "noun2",...},
        #             ...
        #         },
        #          ...
        #     }
        # }


    chain = prompt | llm | output_parser
    # extracted_json = simple_json_extraction(response.content)
    # parsed_response = output_parser.parse(extracted_json)
    response = chain.invoke({"schema":schema, "question":question, "relevant_table_column":parsed_question})
    # print(response)
    clean_sql_query = response.strip('`').replace('sql\n', '', 1).strip()
    if response.strip() == "NOT_ENOUGH_INFO":
        return {"sql_query": "NOT_RELEVANT"}
    else:
        return {"sql_query": clean_sql_query}

In [673]:
generated_sql_que = generete_sql_query({
    "schema":schema,
    "question":question,
    "parsed_question":parse_question 
})

======= generete_sql_query =======


### Validate SQL Query


In [674]:
def validate_and_fix_sql(state: dict) -> dict:
    print("========= validate_and_fix_sql ========")
    """Validate and fix the generated SQL query."""
    sql_query = state['sql_query']
    if sql_query == "NOT_RELEVANT":
        return {"sql_query": "NOT_RELEVANT", "sql_valid": False}
    
    prompt = ChatPromptTemplate.from_messages([
        ("system", '''
    You are an AI assistant that validates and fixes SQL queries. Your task is to:
    1. Check if the SQL query is valid.
    2. Ensure all table and column names are correctly spelled and exist in the schema. All table and column names should be enclosed in backticks, especially if they contain spaces or special characters.
    3. Ensure the SQL query follows proper syntax (e.g., `JOIN`, `WHERE`, and other clauses are used correctly).
    4. Take into account case sensitivity based on the schema.
    5. If there are any issues, fix them and provide the corrected SQL query.
    6. If no issues are found, return the original query.

    Respond in JSON format with the following structure. Only respond with the JSON:
    {{
        "valid": boolean,
        "issues": string or null,
        "corrected_query": string
    }}
    '''),
        ("human", '''===Database schema:
    {schema}

    ===Generated SQL query:
    {sql_query}

    Respond in JSON format with the following structure. Only respond with the JSON:
    {{
        "valid": boolean,
        "issues": string or null,
        "corrected_query": string
    }}

    For example:
    1. {{
        "valid": true,
        "issues": null,
        "corrected_query": "None"
    }}
                
    2. {{
        "valid": false,
        "issues": "Column USERS does not exist",
        "corrected_query": "SELECT * FROM \`users\` WHERE age > 25"
    }}

    3. {{
        "valid": false,
        "issues": "Column names and table names should be enclosed in backticks if they contain spaces or special characters",
        "corrected_query": "SELECT * FROM \`gross income\` WHERE \`age\` > 25"
    }}
                
    '''),
    ])

    # prompt.format(schema=schema, sql_query=sql_query)
    output_parser = JsonOutputParser()

    chain = prompt | llm | output_parser

    result = chain.invoke({"schema":schema, "sql_query":sql_query})
    
    if result["valid"] and result["issues"] is None:
        return {"sql_query": sql_query, "sql_valid": True}
    else:
        return {
            "sql_query": result["corrected_query"],
            "sql_valid": result["valid"],
            "sql_issues": result["issues"]
        }


In [675]:
valid_sql_query = validate_and_fix_sql(generated_sql_que)

========= validate_and_fix_sql ========


In [676]:
query_result = execute_query(valid_sql_query)

### Get answer in sentence

In [677]:
def format_results(state: dict) -> dict:
    print("========= format_results ========")
    """Format query results into a human-readable response."""
    question = state['question']
    results = state['query_result']
    if results == "NOT_RELEVANT":
        return {"answer": "Sorry, I can only give answers relevant to the database."}
    prompt = ChatPromptTemplate.from_messages([
        ("system", '''
    You are an AI assistant that converts database query results into a clear, concise human-readable response. Your goal is to provide a brief conclusion to the user's question based on the query results. 
    Instructions:
    1. Respond in one sentence.
    2. Highlight the key result by enclosing it in double asterisks (**).
    3. Avoid using markdown or unnecessary formatting.

    '''),
        ("human", "User question: {question}\n\nQuery results: {results}\n\nConclusion:")
    ])
    chain = prompt | llm | output_parser
    response = chain.invoke({"question":question, "results":results})
    return {"answer": response}

In [678]:
format_results({
    "question":question, 
    "query_result":query_result
})

========= format_results ========


{'answer': 'The product category generating the most revenue is **beleza_saude**. \n'}

### Format response for visualization

In [705]:
def choose_visualization(state: dict) -> dict:
    print("========= choose_visualization ========")
    """Choose an appropriate visualization for the data."""
    question = state['question']
    results = state['query_result']
    sql_query = state['sql_query']

    if results == "NOT_RELEVANT":
        return {"visualization": "none", "visualization_reasoning": "No visualization needed for irrelevant questions."}
    
    prompt = ChatPromptTemplate.from_messages([
        ("system", '''
    You are an AI assistant recommending the best data visualizations. Based on the user's question, SQL query, and query results, suggest the most suitable graph or chart type.

    ### Chart Types:
    - **Bar Graph**: For comparing categorical data or showing changes over time with more than two categories.
    - **Horizontal Bar Graph**: For comparing few categories or when there's a large disparity between them.
    - **Scatter Plot**: For showing relationships or distributions between two continuous numerical variables.
    - **Pie Chart**: For displaying proportions or percentages of a whole.
    - **Line Graph**: For showing trends over time, where both x and y axes are continuous.
    - **None**: If no visualization is appropriate.

    ### Consider These Questions:
    1. **Aggregations**: Summarize data (e.g., average revenue by month) — Line Graph.
    2. **Comparisons**: Compare metrics (e.g., sales of Product A vs. Product B) — Line or Bar Graph.
    3. **Distributions**: Show data distribution (e.g., age distribution) — Scatter Plot.
    4. **Trends Over Time**: Show changes over time (e.g., website visits) — Line Graph.
    5. **Proportions**: Show percentages (e.g., market share) — Pie Chart.
    6. **Correlations**: Show relationships (e.g., marketing spend vs. revenue) — Scatter Plot.

    ### Format:
         {{
            recommended_visualization: string (bar | horizontal_bar | line | pie | scatter | none),
            reason: Brief explanation of your recommendation
         }}
    '''),
        ("human", '''
    User question: {question}
    SQL query: {sql_query}
    Query results: {results}

    Recommend a visualization:
        '''),
    ])

    output_parser = JsonOutputParser()

    chain = prompt | llm | output_parser

    response = chain.invoke({"question":question,"sql_query":sql_query,"results":results})

    return {
        "recommended_visualization":response["recommended_visualization"],
        "reason":response["reason"]
    }


In [706]:
visualization = choose_visualization({
    "question":question, 
    "query_result":query_result,
    "sql_query":valid_sql_query
})

========= choose_visualization ========


In [707]:
visualization

{'recommended_visualization': 'bar',
 'reason': 'The query compares revenue generated by different product categories. A bar graph effectively visualizes categorical data and allows for easy comparison of revenue values.'}

### Format data for visualization

In [681]:
from abc import ABC, abstractmethod

class BaseFormatter(ABC):
    def __init__(self, llm_manager):
        self.llm = llm_manager

    @abstractmethod
    def format(self, results: list, question: str) -> dict:
        """Formats data for visualization."""
        pass

    def get_axis_label(self, question, data, axis):
        """Get axis label using the LLM."""
        prompt = ChatPromptTemplate.from_messages([
            ("system", f"You are a data labeling expert. Given a question and some data, provide a concise and relevant label for the {{axis}}-axis."),
            ("human", f"Question: {question}\nData : {data}\n\nProvide a concise label for the {axis}-axis."),
        ])
        chain = prompt | llm | output_parser
        return chain.invoke({"question":question, "data":query_result, "axis":"y"})



In [682]:
prompt2 = ChatPromptTemplate.from_messages([
    ("system", f"You are a data labeling expert. Given a question and some data, provide a concise and relevant label for the {{axis}}-axis."),
    ("human", f"Question: {question}\nData : {{data}}\n\nProvide a concise label for the {{axis}}-axis."),
])
chain = prompt2 | llm 
chain.invoke({"question":question, "data":query_result, "axis":"y"})


AIMessage(content='Revenue  \n', response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 1371, 'total_tokens': 1376, 'completion_time': 0.009090909, 'prompt_time': 0.044620864, 'queue_time': 0.002871516999999997, 'total_time': 0.053711773}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-6720a7e6-fc9f-4c7c-86ac-e4a8f271e7a3-0', usage_metadata={'input_tokens': 1371, 'output_tokens': 5, 'total_tokens': 1376})

In [715]:
### Chart formaters

class LineChartFormatter(BaseFormatter):
    def format(self, results: list, question: str) -> dict:
        print("=========Line chart trigger==========")
        if isinstance(results, str):
            results = eval(results)

        x_values = [str(row[0]) for row in results]
        y_values = [float(row[1]) for row in results]

        y_axis_label = self.get_axis_label(question, results[:2], "y")
        
        return {
            "xValues": x_values,
            "yValues": [{"data": y_values, "label": y_axis_label.strip()}],
        }


class ScatterPlotFormatter(BaseFormatter):
    def format(self, results: list, question: str) -> dict:
        print("=========Scatter plot trigger==========")
        if isinstance(results, str):
            results = eval(results)

        series = [{"x": float(x), "y": float(y), "id": i + 1} for i, (x, y) in enumerate(results)]
        return {"series": [{"data": series, "label": "Data Points"}]}


class BarChartFormatter(BaseFormatter):
    def format(self, results: list, question: str) -> dict:
        print("=========Bar chart trigger==========")
        if isinstance(results, str):
            results = eval(results)

        labels = [str(row[0]) for row in results]
        data = [float(row[1]) for row in results]
        # print("=============Trigger==========")
        y_axis_label = self.get_axis_label(question, results[:2], "y")

        return {
            "labels": labels,
            "values": [{"data": data, "label": y_axis_label.strip()}]
        }


In [718]:
class FormatterRegistry:
    def __init__(self, llm_manager):
        self.formatters = {
            "line": LineChartFormatter(llm_manager),
            "scatter": ScatterPlotFormatter(llm_manager),
            "bar": BarChartFormatter(llm_manager),
            # Add other visualizers like "horizontal_bar", "pie", etc.
        }

    def get_formatter(self, visualization):
        print("Selector method triggered")
        return self.formatters.get(visualization, None)


In [721]:
class DataFormatter:
    def __init__(self):
        self.llm_manager = llm
        self.registry = FormatterRegistry(self.llm_manager)

    def format_data_for_visualization(self, state: dict) -> dict:
        print("========= Main method to format data for visualization. ========")
        """Main method to format data for visualization."""
        print(state)
        visualization = state['recommended_visualization']
        results = state['query_result']
        question = state['question']

        formatter = self.registry.get_formatter(visualization)

        if formatter is not None:
            try:
                return {"formatted_data_for_visualization": formatter.format(results, question)}
            except Exception as e:
                return {"error": str(e)}
        
        return {"formatted_data_for_visualization": None}


In [686]:
print(visualization["recommended_visualization"])
visualization_data = DataFormatter().format_data_for_visualization({
    "recommended_visualization": visualization["recommended_visualization"], 
    "query_result": query_result, 
    "question": question
    })
# visualization_data

bar


In [687]:
print(visualization_data)

{'formatted_data_for_visualization': {'labels': ['beleza_saude', 'relogios_presentes', 'cama_mesa_banho', 'esporte_lazer', 'informatica_acessorios', 'moveis_decoracao', 'cool_stuff', 'utilidades_domesticas', 'automotivo', 'ferramentas_jardim', 'brinquedos', 'bebes', 'perfumaria', 'telefonia', 'moveis_escritorio', 'papelaria', 'pcs', 'pet_shop', 'instrumentos_musicais', 'eletroportateis', 'eletronicos', 'consoles_games', 'fashion_bolsas_e_acessorios', 'construcao_ferramentas_construcao', 'malas_acessorios', 'eletrodomesticos_2', 'casa_construcao', 'eletrodomesticos', 'agro_industria_e_comercio', 'moveis_sala', 'telefonia_fixa', 'casa_conforto', 'climatizacao', 'audio', 'portateis_casa_forno_e_cafe', 'livros_interesse_geral', 'moveis_cozinha_area_de_servico_jantar_e_jardim', 'construcao_ferramentas_iluminacao', 'construcao_ferramentas_seguranca', 'industria_comercio_e_negocios', 'alimentos', 'market_place', 'construcao_ferramentas_jardim', 'artes', 'fashion_calcados', 'bebidas', 'sinaliz

### Define state

In [713]:
from typing import List, Any, Annotated, Dict
from typing_extensions import TypedDict
import operator

class InputState(TypedDict):
    question: str
    schema: List[Dict]
    parsed_question: dict
    sql_query: str
    sql_valid: bool
    sql_issues: str
    query_result: List[Any]
    recommended_visualization:str
    reason:str
    visualization: Annotated[str, operator.add]
    # visualization_reason: Annotated[str, operator.add]
    # formatted_data_for_visualization: Dict[str, Any]

class OutputState(TypedDict):
    parsed_question: Dict[str, Any]
    unique_nouns: List[str]
    sql_query: str
    sql_valid: bool
    sql_issues: str
    query_result: List[Any]
    recommended_visualization:str
    reason:str
    results: List[Any]
    answer: Annotated[str, operator.add]
    error: str
    visualization: Annotated[str, operator.add]
    visualization_reason: Annotated[str, operator.add]
    formatted_data_for_visualization: Dict[str, Any]

In [729]:
from langgraph.graph import StateGraph

from langgraph.graph import START, END

class WorkflowManager:
    def __init__(self):
        self.data_formatter = DataFormatter()

    def create_workflow(self) -> StateGraph:
        """Create and configure the workflow graph."""
        workflow = StateGraph(input=InputState, output=OutputState)

        # Add nodes to the graph
        workflow.add_node("parse_question", get_parse_question)
        workflow.add_node("get_unique_nouns", get_unique_nouns)
        workflow.add_node("generate_sql", generete_sql_query)
        workflow.add_node("validate_and_fix_sql", validate_and_fix_sql)
        workflow.add_node("execute_sql", execute_query)
        workflow.add_node("format_results", format_results)
        workflow.add_node("choose_visualization", choose_visualization)
        workflow.add_node("format_data_for_visualization", self.data_formatter.format_data_for_visualization)
        
        # Define edges
        workflow.add_edge(START, "parse_question")
        workflow.add_edge("parse_question", "get_unique_nouns")
        workflow.add_edge("get_unique_nouns", "generate_sql")
        workflow.add_edge("generate_sql", "validate_and_fix_sql")
        workflow.add_edge("validate_and_fix_sql", "execute_sql")
        workflow.add_edge("execute_sql", "format_results")
        workflow.add_edge("execute_sql", "choose_visualization")
        workflow.add_edge("choose_visualization", "format_data_for_visualization")
        workflow.add_edge("format_data_for_visualization", END)
        workflow.add_edge("format_results", END)
        # workflow.set_entry_point("parse_question")

        return workflow
    
    def returnGraph(self):
        return self.create_workflow().compile()

    def run_sql_agent(self, question: str, schema: List[Dict]) -> dict:
        """Run the SQL agent workflow and return the formatted answer and visualization recommendation."""
        app = self.create_workflow().compile()
        for event in app.stream({"question": question, "schema": schema}):
            for value in event.values():
                print(value)

        # result = app.invoke({"question": question, "schema": schema})
        # print(result)
        # return {
        #     "answer": result['answer'],
        #     "visualization": result['visualization'],
        #     "visualization_reason": result['visualization_reason'],
        #     "formatted_data_for_visualization": result['formatted_data_for_visualization']
        # }

In [730]:
workflow = WorkflowManager()
workflow.run_sql_agent(
    schema=schema,question="Which product categories generate the most revenue?"
)
# app = workflow.returnGraph()

/var/folders/fq/21nt5n9d46bgh23zzhvzdgk00000gn/T/ipykernel_66944/2722828617.py:11: LangGraphDeprecationWarning: Initializing StateGraph without state_schema is deprecated. Please pass in an explicit state_schema instead of just an input and output schema.
  workflow = StateGraph(input=InputState, output=OutputState)


{'question': 'Which product categories generate the most revenue?', 'schema': [{'table_name': 'olist_products_dataset', 'schema': [{'name': 'product_id', 'type': 'TEXT', 'nullable': True}, {'name': 'product_category_name', 'type': 'TEXT', 'nullable': True}, {'name': 'product_name_lenght', 'type': 'FLOAT', 'nullable': True}, {'name': 'product_description_lenght', 'type': 'FLOAT', 'nullable': True}, {'name': 'product_photos_qty', 'type': 'FLOAT', 'nullable': True}, {'name': 'product_weight_g', 'type': 'FLOAT', 'nullable': True}, {'name': 'product_length_cm', 'type': 'FLOAT', 'nullable': True}, {'name': 'product_height_cm', 'type': 'FLOAT', 'nullable': True}, {'name': 'product_width_cm', 'type': 'FLOAT', 'nullable': True}]}, {'table_name': 'olist_orders_dataset', 'schema': [{'name': 'order_id', 'type': 'TEXT', 'nullable': True}, {'name': 'customer_id', 'type': 'TEXT', 'nullable': True}, {'name': 'order_status', 'type': 'TEXT', 'nullable': True}, {'name': 'order_purchase_timestamp', 'type'

In [724]:
app = workflow.returnGraph()
from IPython.display import Image, display

try:
    display(Image(app.get_graph().draw_mermaid_png()))
except Exception:
    # This requires some extra dependencies and is optional
    pass

/var/folders/fq/21nt5n9d46bgh23zzhvzdgk00000gn/T/ipykernel_66944/1950226658.py:11: LangGraphDeprecationWarning: Initializing StateGraph without state_schema is deprecated. Please pass in an explicit state_schema instead of just an input and output schema.
  workflow = StateGraph(input=InputState, output=OutputState)


/var/folders/fq/21nt5n9d46bgh23zzhvzdgk00000gn/T/ipykernel_66944/70840226.py:11: LangGraphDeprecationWarning: Initializing StateGraph without state_schema is deprecated. Please pass in an explicit state_schema instead of just an input and output schema.
  workflow = StateGraph(input=InputState, output=OutputState)


KeyError: 'schema'